In [2]:
import numpy as np
import pandas as pd
from scipy import linalg


>아래 내용"만" 참고하여 다음 업무를 수행할 것
- Matrix Factorization 을 numpy 만으로 구현할 것
- 영화 시청 이력을 기반으로 영화 평점 예측 모델 구현
- 데이터 셋은 아래 것을 사용할 것: https://grouplens.org/datasets/movielens/
- 학습 데이터:  timestamp >= 1104505203 ~ timestamp <= 1230735592
- 평가 데이터: timestamp >= 1230735600 이후
- 최종 산출물 평가데이터에 대한 아래 결과와 함께 구현 코드: $<userId>,<movieId>,<predicted rating>,<timestamp>$
- 구현의 성능 향상을 위한 동시성 사용방안에 대해 고민할 것

https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf?fbclid=IwAR1Ff0kNw5Q8FcAlseaifAa6FQ4e4CynoKEFW4yMWrSCDdnJhLGEEv2vtKc

https://en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems)#:~:text=Matrix%20factorization%20is%20a%20class,two%20lower%20dimensionality%20rectangular%20matrices.

https://www.youtube.com/watch?v=ZspR5PZemcs&feature=youtu.be&fbclid=IwAR1s02b72t_kUUXKRmvcJZMjEwAXlHhrTdVKMJr8svuyTXSgOI17hrb8_rE


>data set
* movies.csv
* ratings.csv



In [160]:
df_scores = pd.read_csv("genome-scores.csv")
df_scores

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075
...,...,...,...
15584443,206499,1124,0.11000
15584444,206499,1125,0.04850
15584445,206499,1126,0.01325
15584446,206499,1127,0.14025


In [162]:
df_tag = pd.read_csv("genome-tags.csv")
df_tag

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [161]:
df_link =pd.read_csv("links.csv")
df_link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
62418,209157,6671244,499546.0
62419,209159,297986,63407.0
62420,209163,6755366,553036.0
62421,209169,249603,162892.0


In [3]:
df_ratings = pd.read_csv("ratings.csv")

In [4]:
df_ratings.head() #userId, movieId, rating, timestamp

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
df_all = df_ratings.copy()

In [13]:
df_train = df_all[df_all["timestamp"] >= 1104505203]

In [15]:
df_train = df_train[df_train["timestamp"] <= 1230735592]

In [18]:
df_train.shape[0] / df_all.shape[0]

0.18408354048254616

In [19]:
df_test = df_all[df_all["timestamp"] >= 1230735600]

In [20]:
df_test.shape[0] / df_all.shape[0]

0.4589131361300827

df_train , df_test 나눔

In [51]:
df_movies = pd.read_csv("movies.csv") # movieid

In [52]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [24]:
df_movies.genres.unique().shape

(1639,)

>genres
   * Action
   * Adventure
   * Animation
   * Children's
   * Comedy    
   * Crime
   * Documentary
   * Drama
   * Fantasy
   * Film-Noir
   * Horror
   * Musical
   * Mystery
   * Romance
   * Sci-Fi
   * Thriller
   * War
   * Western
   * (no genres listed)


In [53]:
df_movies['genres'] = df_movies['genres'].apply(lambda x : x.split("|"))

In [54]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [55]:
df = df_movies.copy()

In [56]:
from sklearn.preprocessing import MultiLabelBinarizer

In [57]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                         columns = mlb.classes_,
                         index = df.index))

In [58]:
df.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
del df['title']

In [121]:
# category (영화, 장르)
df.tail()

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
62418,209157,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
62419,209159,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
62420,209163,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
62421,209169,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62422,209171,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [66]:
df.shape

(62423, 21)

In [80]:
# 유저, 영화, 평점
df_train.head()


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
24997287,162521,56174,4.0,1219357742
24997298,162521,59315,4.5,1219339433
24997301,162521,59615,3.5,1219339443
24997302,162521,59725,4.0,1219339392


In [85]:
df_train_piv = df_train.pivot("userId","movieId","rating")
df_train_piv = df_train_piv.fillna(0)
df_train_piv.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
df_train_piv.shape

(27915, 10533)

In [148]:
# user의 rating 평균
user_mean = np.mean(df_train_piv, axis = 1)
user_mean

userId
1         0.025349
2         0.063420
10        0.017374
11        0.007168
12        0.128691
            ...   
162511    0.012485
162512    0.116064
162513    0.031757
162516    0.832526
162521    0.075667
Length: 27915, dtype: float64

In [90]:
df_train_piv.shape

(27915, 10533)

In [149]:
piv_mean = df_train_piv - user_mean.reshape(-1,1)

AttributeError: 'Series' object has no attribute 'reshape'

### SVD ?

In [87]:
U,s,VT = linalg.svd(df_train_piv,compute_uv = True  )

In [88]:
r = s.shape[0] - sum(np.allclose(lx,0) for lx in s)
reduced = U[:,:r]*s[:r]@VT[:r,:] 
reduced

array([[ 1.23243723e-15, -4.51086518e-15,  2.04546032e-15, ...,
        -2.05001031e-15, -1.24851979e-15,  1.88795174e-17],
       [ 3.50000000e+00, -2.42742319e-14, -2.31132420e-14, ...,
        -6.36020946e-16,  1.10439544e-16, -5.49012875e-17],
       [ 3.50000000e+00, -4.01292939e-15,  2.16728515e-14, ...,
        -1.29228847e-15,  1.24109809e-16, -3.70255042e-17],
       ...,
       [ 2.19620820e-16, -2.64763970e-18, -5.06239223e-15, ...,
        -3.33361675e-16, -1.21430643e-16,  5.17503249e-17],
       [ 4.50000000e+00,  2.50000000e+00,  5.00000000e-01, ...,
        -1.84564244e-15, -6.65493034e-16,  9.96890016e-17],
       [ 4.00000000e+00, -5.48199723e-18, -3.72006871e-15, ...,
        -1.99883767e-15, -2.13155841e-18,  9.56808417e-17]])

In [89]:
reduced.shape

(27915, 10533)

In [94]:
U.shape

(27915, 27915)

In [96]:
VT.shape

(10533, 10533)

### Least-square-solution?
Ax = b

In [138]:
y_train = df_train["rating"]
y_train.head()

0    5.0
1    3.5
2    5.0
3    5.0
4    3.5
Name: rating, dtype: float64

In [101]:
df_train

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
24997287,162521,56174,4.0,1219357742
24997298,162521,59315,4.5,1219339433
24997301,162521,59615,3.5,1219339443
24997302,162521,59725,4.0,1219339392


In [103]:
x_train = df_train[["userId","movieId"]]
x_train.head()

,userId,movieId
0,1,296
1,1,306
2,1,307
3,1,665
4,1,899


In [128]:
print(df.iloc[302])

movieId               306
(no genres listed)      0
Action                  0
Adventure               0
Animation               0
Children                0
Comedy                  0
Crime                   0
Documentary             0
Drama                   1
Fantasy                 0
Film-Noir               0
Horror                  0
IMAX                    0
Musical                 0
Mystery                 0
Romance                 0
Sci-Fi                  0
Thriller                0
War                     0
Western                 0
Name: 302, dtype: int64


In [106]:
df_x = pd.merge(x_train,df, how='left', on='movieId')
df_x.head()


,userId,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,296,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,306,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,307,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,665,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,899,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [143]:
x_hat,res,rank,s = linalg.lstsq(x_train,y_train,cond=None)

In [144]:
rank,res

(2, 17488610.82919692)

In [159]:
Ax = x_train@x_hat
Ax

0           0.010324
1           0.010672
2           0.010707
3           0.023157
4           0.031295
              ...   
24997287    6.766479
24997298    6.875717
24997301    6.886151
24997302    6.889977
24997308    6.902114
Length: 4602106, dtype: float64

In [157]:
y_train

0           5.0
1           3.5
2           5.0
3           5.0
4           3.5
           ... 
24997287    4.0
24997298    4.5
24997301    3.5
24997302    4.0
24997308    4.0
Name: rating, Length: 4602106, dtype: float64